In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\jaesc2\GitHub\skforecast


In [5]:
import re
import pytest
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from skforecast.exceptions import ResidualsUsageWarning
from skforecast.direct import ForecasterDirectMultiVariate

# Fixtures
# from skforecast.direct.tests.tests_forecaster_direct.fixtures_forecaster_direct import y
# from skforecast.direct.tests.tests_forecaster_direct.fixtures_forecaster_direct import exog, exog_predict
from skforecast.direct.tests.tests_forecaster_direct_multivariate.fixtures_forecaster_direct_multivariate import series
from skforecast.direct.tests.tests_forecaster_direct_multivariate.fixtures_forecaster_direct_multivariate import exog, exog_predict


transformer_exog = ColumnTransformer(
                       [('scale', StandardScaler(), ['exog_1']),
                        ('onehot', OneHotEncoder(), ['exog_2'])],
                       remainder = 'passthrough',
                       verbose_feature_names_out = False
                   )

In [17]:
forecaster = ForecasterDirectMultiVariate(
                     regressor          = LinearRegression(),
                     level              = 'l1',
                     steps              = 3,
                     lags               = 3,
                     transformer_series = StandardScaler()
                 )
forecaster.fit(series=series)
results = forecaster.predict_interval(
    steps=3, method='conformal', interval=0.95, use_binned_residuals=True
)

In [18]:
results

,level,pred,lower_bound,upper_bound
50,l1,0.631143,0.176033,1.086252
51,l1,0.380042,0.128327,0.631757
52,l1,0.332560,0.080845,0.584275


In [19]:
results[['pred', 'lower_bound', 'upper_bound']].to_numpy()

array([[0.63114259, 0.17603311, 1.08625208],
       [0.3800417 , 0.12832655, 0.63175685],
       [0.33255977, 0.08084462, 0.58427492]])